# Demo notebook for Kamodo Flythrough Coordinates and Plotting functions
This notebook tutorial shows how to retrieve a real trajectory through SSCWeb, create a sample trajectory in GDZ spherical coordinates, convert a given trajectory to any coordinate system supported by SpacePy and most coordinate systems supported by AstroPy, and produce a variety of interactive plots.

In [ ]:
# Import function to retrieve a sample trajectory from the grace1 satellite via SSCWeb.
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
help(SF.SatelliteTrajectory)

In [ ]:
# Import satellite flythrough coordinate conversion code for more info on the coordinate systems.
from kamodo_ccmc.flythrough.utils import ConvertCoord
help(ConvertCoord)

In [ ]:
# Retrieve a real satellite trajectory.
# Typical coordinates possible through SSCWeb are GEO, GSE, SM, and GSM (all cartesian and in R_E).
traj_dict, coord_type = SF.SatelliteTrajectory('grace1', 1426660000.0, 1426680700.0, coord_type='GEO')
traj_hrs = (traj_dict['sat_time']-traj_dict['sat_time'].min())/3600.  #convert to hours since interval start
print(coord_type)
print('sat_time=utc timestamp:',traj_dict['sat_time'][:10])
print('c1=x(R_E):',traj_dict['c1'][:10])
print('c2=y(R_E):',traj_dict['c2'][:10])
print('c3=z(R_E):',traj_dict['c3'][:10])

In [ ]:
# Show grace1 trajectory in given coordinate system for different plot types.
# The first example is a pair of plots: a 4D plot and a stack of 4 time-series plots.
from kamodo_ccmc.flythrough.plots import SatPlot4D
help(SatPlot4D)

In [ ]:
# Using time since the start as the variable values to be plotted. Keeping original coordinate system (GEO).
# Showing 3D plot. Remember that the body='earth' option is only accurate/possible in the GEO coordinate system.
SatPlot4D('utc_time', traj_dict['sat_time'], traj_dict['c1'], traj_dict['c2'], traj_dict['c3'],
          traj_hrs, 'hr', *coord_type.split('-'), 'GSE', 'all', 'grace1', body='earth', type='2DPN')

In [ ]:
# Another way to get a trajectory is via the SampleTrajectory flythrough function
help(SF.SampleTrajectory)

In [ ]:
# Get a sample trajectory.
sample_traj, sample_coord = SF.SampleTrajectory(1426660000.0, 1426680700.0, max_height=550., 
                                                             min_height=525., p=0.05, n=30.)
sample_hrs = (sample_traj['sat_time']-sample_traj['sat_time'].min())/3600.  # convert to hours since interval start
sample_traj.keys(), sample_coord

In [ ]:
# Convert coordinates to another coordinate system, this time an AstroPy coordinate system.
c1, c2, c3, units = ConvertCoord(sample_traj['sat_time'], sample_traj['c1'], sample_traj['c2'], sample_traj['c3'], 
                                 *sample_coord.split('-'), 'teme', 'car')
print('c1=longitude:',c1[:10])
print('c2=latitude:',c2[:10])
print('c3=radius:',c3[:10])
print('units (c1,c2,c3):',units)

In [ ]:
# Using time since the start as the variable values to be plotted. Converting into a different coordinate system in
#  AstroPy instead of the given coordinate system.
# Showing 1D stack of plots.
SatPlot4D('utc_time', sample_traj['sat_time'], c1, c2, c3, sample_hrs, 'hr', 
          'teme', 'car', 'geocentrictrueecliptic', 'all', 'sample', plotCoordType1D='sph', type='1D')

In [ ]:
# How to get a trajectory from TLEs
import datetime as dt
tle_file = 'C:/Users/rringuet/Kamodo_Data/TIEGCM/Uriel_Ramirez_012517_IT_1/GRACE1_TLEs.txt'
time_cadence = 60.  # seconds between propagated trajectory positions
start_utcts = dt.datetime(2015, 3, 18, 0, 20).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2015, 3, 21, 0, 0).replace(tzinfo=dt.timezone.utc).timestamp()
traj_dict, coord_type = SF.TLETrajectory(tle_file, start_utcts, end_utcts, time_cadence)
traj_dict

In [ ]:
# Plotting all of the trajectory with time as the color in 3D
traj_hrs = (traj_dict['sat_time']-traj_dict['sat_time'].min())/3600.  # convert to hours since interval start
SatPlot4D('utc_time', traj_dict['sat_time'], traj_dict['c1'], traj_dict['c2'], traj_dict['c3'],
          traj_hrs, 'hr', *coord_type.split('-'), 'GEO', 'all', 'grace1', body='earth', type='3D')